In [1]:
from db_handler import DBHandler
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [17]:
db = DBHandler()
db.setup_test_table()

X_train_list = []
y_train_list = []
for x in db.read_data("comp3208-train-small.csv"):
    listParts = x.strip().split(',')
    if listParts[2] == "rating":
        continue
    X_train_list.append([listParts[0], listParts[1], listParts[3]])
    y_train_list.append(listParts[2])

X_train = np.array(X_train_list)
y_train = np.array(y_train_list)


INFO 2021-02-18 17:48:24,165 logging started
INFO 2021-02-18 17:48:24,171 Setting up test table
INFO 2021-02-18 17:48:26,337 Reading data from database: comp3208-train-small.csv


In [6]:
X_test_list = []
for x in db.read_data("comp3208-test-small.csv"):
    listParts = x.strip().split(',')
    X_test_list.append([listParts[0], listParts[1], listParts[2]])

X_test = np.array(X_test_list)

INFO 2021-02-18 17:37:59,703 Reading data from database: comp3208-test-small.csv


In [21]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=1000)

tree.fit(X_train, y_train)

prediction = tree.predict(X_train)

In [22]:
from sklearn.metrics import mean_squared_error

prediction = prediction.astype(np.float64)

mse = mean_squared_error(y_train.astype(np.float64), prediction)

In [23]:
print(mse)

0.0
